## Cryptocurrency trading bot.

### Install if needed.

In [1]:
##%pip install python-binance==0.7.5
#%pip install --upgrade python-binance
#%pip install paramiko

### Prelude.

In [2]:
# Import libraries.
from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.exchange import Cryptocurrency_exchange
from cryptocurrency.conversion import get_timezone_offset_in_seconds
from cryptocurrency.conversion_table import get_conversion_table

# Manage API keys.
authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
client = authenticator.spot_client

# Get all available pair information for trading.
exchange = Cryptocurrency_exchange(client=client, directory='crypto_logs')
exchange_info = exchange.info

# Precalculate UTC offset for inter-server communication coherence.
offset_s = get_timezone_offset_in_seconds()

# Precalculate conversion_table.
conversion_table = get_conversion_table(client=client, 
                                        exchange_info=exchange_info, 
                                        offset_s=offset_s, 
                                        dump_raw=False, 
                                        as_pair=True, 
                                        minimal=False, 
                                        extra_minimal=False, 
                                        super_extra_minimal=False, 
                                        convert_to_USDT=False)

# Display pre-computed conversion table.
conversion_table

,symbol,price_change,price_change_percent,weighted_average_price,close_shifted,close,last_volume,bid_price,bid_volume,ask_price,...,rolling_quote_volume,open_time,first_ID,last_ID,count,base_asset,quote_asset,rolling_base_quote_volume,bid_ask_percent_change,bid_ask_volume_percent_change
date,,,,,,,,,,,,,,,,,,,,,
2022-12-20 22:20:05.949,CAKEGBP,0.12500000,4.536,2.83483525,2.750000,2.881000,77.87000000,2.830000,68.4600,2.879000,...,5.210144e+02,1671488405949,593824,593834,11,CAKE,GBP,1.808450e+02,1.701980,44.142111
2022-12-20 22:36:23.914,FISBRL,0.02000000,1.600,1.27513981,1.250000,1.270000,90.20000000,1.271000,860.3000,1.291000,...,1.452703e+04,1671489383914,388230,388264,35,FIS,BRL,1.143861e+04,1.549187,86.688835
2022-12-20 22:43:15.555,SLPBIDR,1.20,3.692,33.40,32.700000,33.700000,2000.00000000,33.700000,1056.0000,34.100000,...,2.393147e+08,1671489795555,1104576,1104783,208,SLP,BIDR,7.101325e+06,1.173021,6.683121
2022-12-20 22:44:24.578,ANCBNB,0.00000050,0.357,0.00014548,0.000142,0.000140,10000.00000000,0.000140,907.2800,0.000141,...,3.353366e+01,1671489864578,277670,277861,192,ANC,BNB,2.386738e+05,0.777385,27.344925
2022-12-20 22:44:30.713,ALCXBUSD,0.60000000,4.225,14.85823383,14.100000,14.800000,88.82510000,14.800000,38.2284,14.900000,...,1.599360e+05,1671489870713,879903,880636,734,ALCX,BUSD,1.080649e+04,0.671141,33.052652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-20 23:03:46.406,DOGEBTC,0.00000010,2.304,0.00000440,0.000004,0.000004,534.00000000,0.000004,92475.0000,0.000004,...,2.028512e+02,1671491026406,59848568,59859765,11198,DOGE,BTC,4.568721e+07,0.225225,9.998930
2022-12-20 23:03:46.406,DOGEBUSD,0.00365000,5.124,0.07380500,0.071240,0.074890,41228.00000000,0.074880,13640.0000,0.074890,...,3.451015e+07,1671491026406,90602097,90688773,86677,DOGE,BUSD,4.608112e+08,0.013353,10.351056
2022-12-20 23:03:46.406,ZENUSDT,0.45000000,5.455,8.53548322,8.260000,8.700000,45.85000000,8.690000,726.3500,8.700000,...,5.690124e+05,1671491026406,36087208,36093118,5911,ZEN,USDT,6.540372e+04,0.114943,99.844669


### This is how to synchronize the wallet with the virtual wallet (take all above zero).

In [3]:
from cryptocurrency.trader.wallet import select_asset_with_biggest_wallet
from_asset, converted_quantity, quantity = \
    select_asset_with_biggest_wallet(client=client, 
                                     conversion_table=conversion_table, 
                                     exchange_info=exchange_info)
priority = 'fees' if float(converted_quantity) > 10.0 else 'wallet'
from_asset, converted_quantity, quantity

('BUSD', '38.8286', 38.82861119)

### Trade.

In [4]:
to_asset = 'BTC'

from cryptocurrency.trader.trade import trade
import pandas as pd

conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
request = trade(client=client, to_asset=to_asset, conversion_table=conversion_table, 
                exchange_info=exchange_info, priority=priority)
from_asset = to_asset
request

[('BTC', 'BUSD')]


{'symbol': 'BTCBUSD',
 'orderId': 7709507542,
 'orderListId': -1,
 'clientOrderId': 'CKDDCk4qxdw5QcCc0KiIb0',
 'transactTime': 1671577430658,
 'price': '0.00000000',
 'origQty': '0.00229000',
 'executedQty': '0.00229000',
 'cummulativeQuoteQty': '38.70699980',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'BUY',
 'workingTime': 1671577430658,
 'fills': [{'price': '16902.62000000',
   'qty': '0.00229000',
   'commission': '0.00000000',
   'commissionAsset': 'BNB',
   'tradeId': 707337743}],
 'selfTradePreventionMode': 'NONE'}

### Main loop buying screened assets in pairs which are connected to the wallet holdings.

In [ ]:
sell_asset = 'BUSD'
take_profit = None
stop_loss = None
purchased_asset = None
asset_to_delete = None
keys_file = 'server_keys.txt'
input_log = '~/workspace/crypto_logs/crypto_input_log_15s.txt'
output_log_screened = '~/workspace/crypto_logs/crypto_output_log_1d_screened.txt'

from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.exchange import Cryptocurrency_exchange
from cryptocurrency.conversion import select_pair_with_highest_quote_volume_from_base_asset
from cryptocurrency.trader.ssh import Ssh
from cryptocurrency.trader.wallet import select_asset_with_biggest_wallet
from cryptocurrency.trader.order_book import get_order_book_trigger
from cryptocurrency.trader.trade import trade
import time

authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
client = authenticator.spot_client

ssh = Ssh(input_log=input_log, output_log_screened=output_log_screened, keys_file=keys_file)

def choose_to_asset(from_asset, to_asset, latest_asset, output_log_screened=output_log_screened):
    tradable_pairs = ssh.get_logs_from_server(server_log=ssh.output_log_screened)
    if tradable_pairs.shape[0] < 1:
        to_asset = sell_asset
    else:
        print('.', end='')
        tradable_pairs = tradable_pairs.sort_values(by='last_price_move', ascending=False)
        tradable_assets = list(set(tradable_pairs['symbol'].tolist()))
        if from_asset in tradable_assets:
            to_asset = latest_asset = from_asset
        else:
            buyable = False
            for test_asset in tradable_assets:
                test_symbol = \
                    select_pair_with_highest_quote_volume_from_base_asset(base_asset=test_asset, 
                                                                          conversion_table=conversion_table, 
                                                                          exchange_info=exchange_info)
                time.sleep(1.0)
                if get_order_book_trigger(client=client, symbol=test_symbol, threshold=10000):
                    latest_asset = test_asset
                    buyable = True
                    break
            to_asset = latest_asset if buyable else sell_asset
    return latest_asset, to_asset

def trade_conditionally(client, exchange_info, to_asset):
    #conversion_table = ssh.get_logs_from_server(server_log=ssh.input_log)
    conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
    from_asset, converted_quantity, quantity = \
        select_asset_with_biggest_wallet(client=client, conversion_table=conversion_table, 
                                         exchange_info=exchange_info)
    priority = 'fees' if float(converted_quantity) > 10.0 else 'wallet'
    request = trade(client=client, to_asset=to_asset, conversion_table=conversion_table, 
                    exchange_info=exchange_info, priority=priority)
    if request is not None:
        if to_asset != sell_asset:
            purchased_asset = request['symbol']
            price_when_bought = float(request['fills'][0]['price'])
        else:
            purchased_asset = price_when_bought = None
        from_asset = to_asset
    else:
        purchased_asset = price_when_bought = None
    return from_asset, to_asset, purchased_asset, price_when_bought

latest_asset = from_asset
price_when_bought = quantity

while True:
    latest_asset, to_asset = choose_to_asset(from_asset, to_asset, latest_asset, 
                                             output_log_screened=output_log_screened)
    if from_asset != to_asset:
        from_asset, to_asset, purchased_asset, price_when_bought = trade_conditionally(client=client, 
                                                                                       exchange_info=exchange_info, 
                                                                                       to_asset=to_asset)
    time.sleep(2)

ssh.close()

[('BTC', 'BUSD')]
.[('LSK', 'BUSD')]
....[('LSK', 'BUSD')]
.[('LSK', 'BUSD')]
.....

### Rough draft from the older Take Profits/Stop Loss implementation of the main above.

In [ ]:
sell_asset = 'BUSD'
take_profit = 10.0
stop_loss = None
purchased_asset = None
asset_to_delete = None
keys_file = 'server_keys.txt'
input_log = '~/workspace/crypto_logs/crypto_input_log_15s.txt'
output_log_screened = '~/workspace/crypto_logs/crypto_output_log_1d_screened.txt'

from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.exchange import Cryptocurrency_exchange
from cryptocurrency.conversion import select_pair_with_highest_quote_volume_from_base_asset
from cryptocurrency.trader.ssh import Ssh
from cryptocurrency.trader.wallet import select_asset_with_biggest_wallet
from cryptocurrency.trader.order_book import get_order_book_trigger
from cryptocurrency.trader.trade import trade
import time
import pandas as pd

ssh = Ssh(input_log=input_log, output_log_screened=output_log_screened, keys_file=keys_file)

def choose_to_asset(from_asset, to_asset, latest_asset, output_log_screened=output_log_screened):
    tradable_pairs = ssh.get_logs_from_server(server_log=ssh.output_log_screened)
    if tradable_pairs.shape[0] < 1:
        to_asset = sell_asset
    else:
        print('.', end='')
        tradable_pairs = tradable_pairs.sort_values(by='last_price_move', ascending=False)
        tradable_assets = list(set(tradable_pairs['symbol'].tolist()))
        if from_asset in tradable_assets:
            to_asset = latest_asset = from_asset
        else:
            buyable = False
            for test_asset in tradable_assets:
                test_symbol = \
                    select_pair_with_highest_quote_volume_from_base_asset(base_asset=test_asset, 
                                                                          conversion_table=conversion_table, 
                                                                          exchange_info=exchange_info)
                time.sleep(1.0)
                if get_order_book_trigger(client=client, symbol=test_symbol, threshold=10000):
                    latest_asset = test_asset
                    buyable = True
                    break
            to_asset = latest_asset if buyable else sell_asset
    return latest_asset, to_asset

def trade_conditionally(client, exchange_info, to_asset):
    #conversion_table = ssh.get_logs_from_server(server_log=ssh.input_log)
    conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
    from_asset, converted_quantity, quantity = \
        select_asset_with_biggest_wallet(client=client, conversion_table=conversion_table, 
                                         exchange_info=exchange_info)
    priority = 'fees' if float(converted_quantity) > 10.0 else 'wallet'
    request = trade(client=client, to_asset=to_asset, conversion_table=conversion_table, 
                    exchange_info=exchange_info, priority=priority)
    if request is not None:
        if to_asset != sell_asset:
            purchased_asset = request['symbol']
            price_when_bought = float(request['fills'][0]['price'])
        else:
            purchased_asset = price_when_bought = None
        from_asset = to_asset
    else:
        purchased_asset = price_when_bought = None
    return from_asset, to_asset, purchased_asset, price_when_bought

latest_asset = from_asset
price_when_bought = quantity

blacklist = pd.DataFrame(columns=['date', 'asset', 'counter_1', 'counter_2'])

while True:
    try:
        if asset_to_delete is None:
            latest_asset, to_asset = choose_to_asset(from_asset, to_asset, latest_asset, 
                                                     output_log_screened=output_log_screened)
            if from_asset != to_asset:
                from_asset, to_asset, purchased_asset, price_when_bought = \
                    trade_conditionally(client=client, exchange_info=exchange_info, to_asset=to_asset)
            elif purchased_asset is not None:
                #conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
                conversion_table = ssh.get_logs_from_server(server_log=ssh.input_log)
                price_now = conversion_table[conversion_table['symbol'] == purchased_asset]['close'].iat[0]
                percent_gain = ((price_now - price_when_bought) / price_when_bought) * 100.0
                if stop_loss is not None:
                    if percent_gain <= -stop_loss:
                        to_asset = latest_asset
                        asset_to_delete = purchased_asset
                elif take_profit is not None:
                    if percent_gain >= take_profit:
                        to_asset = latest_asset
                        asset_to_delete = purchased_asset
        else:
            connected_pairs = exchange_info[exchange_info['base_asset'] == asset_to_delete]['symbol']
            input_pairs = pd.read_csv(crypto_input_log_screened, index_col=0)
            input_pairs = input_pairs[~(input_pairs['symbol'].isin(connected_pairs))]
            input_pairs.to_csv(crypto_input_log_screened)
            asset_to_delete = purchased_asset = None
        time.sleep(2)
    except (IndexError, pd.errors.EmptyDataError):
        time.sleep(0.5)
        continue

ssh.close()